## LangChain範例-對話(Chat)
2025/06/29 蘇彥庭

1. 單輪對話(Single-turn Chat)

In [15]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

model = init_chat_model("gemma3:1b", model_provider="ollama")

messages = [
    HumanMessage("你是誰?"),
]

res = model.invoke(messages)

print(res.content)

我是 Gemma，一個由 Google DeepMind 訓練的大型語言模型。我是一個開放權務的大型語言模型，可以接收文字和圖片作為輸入，並以文本生成回答你。

簡單來說，我是一個 AI 助手，可以幫你寫作，翻譯，回答問題，以及進行很多其他的事情。

你有什么想和我聊的嗎？ 😊



2. 系統訊息控制(System Message Control)
* SystemMessage
    * 用途：提供系統角色設定或對模型行為的指示。
    * 常見用法：用來告訴模型應該扮演什麼樣的角色或遵循哪些規則。
* HumanMessage
    * 用途：模擬使用者的輸入。
    * 常見用法：用來傳遞使用者的問題或請求。
* AIMessage
    * 用途：模擬 AI 的回應。
    * 常見用法：用來記錄對話中 AI 所回覆的內容，讓後續對話有上下文。

In [9]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

model = init_chat_model("gemma3:1b", model_provider="ollama")

messages = [
    SystemMessage("Translate the following from Chinese into English"),
    HumanMessage("你是誰?"),
]

res = model.invoke(messages)

print(res.content)

You are...? 

It’s a very common and polite way to say “Who are you?” in Chinese. 😊 

It literally translates to “Who are you?” but it’s often used as a greeting or a casual way to ask someone’s name. 

Let me know if you’d like me to translate a specific phrase or sentence!


3 具記憶的多輪對話(Multi-turn Chat with Memory)

3.1 多輪對話無記憶性範例

In [54]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

model = init_chat_model("gemma3:1b", model_provider="ollama")

messages = [
    HumanMessage(content="我叫做王小明，請你記住我的名字!"),
]

res = model.invoke(messages)

print("\n---------- 第1次回答 ----------\n")
print(res.content)


messages = [
    HumanMessage(content="請問我叫做什麼名字?")
]

res = model.invoke(messages)

print("\n---------- 第2次回答 ----------\n")
print(res.content)


---------- 第1次回答 ----------

好的，王小明，我记住了。 😊


---------- 第2次回答 ----------

我是一個大型語言模型，沒有名字。我是由 Google 開發的。你可以叫我“AI”或“助手”。 

你是在用什麼樣的程式或工具與我互動呢？😊



3.2 對話具記憶性範例

In [53]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

model = init_chat_model("gemma3:1b", model_provider="ollama")

messages = [
    HumanMessage(content="我叫做王小明，請你記住我的名字!"),
    AIMessage(content="好，王小明，我記住你！"),  # 模擬 AI 回應 幫助模型記住前一輪內容
    HumanMessage(content="請問我叫做什麼名字?"),  # 新一輪輸入
]

res = model.invoke(messages)

print(res.content)

你叫做小明。😊



3.3 用 ConversationBufferMemory 建立帶記憶的 Chat 流程

In [51]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langchain.memory import ConversationBufferMemory

# 建立 Ollama 模型
llm = init_chat_model("gemma3:1b", model_provider="ollama")

# 建立記憶體
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history",
)

# 手動模擬對話輪次
def ask(input_text):
    # 載入記憶訊息
    history = memory.chat_memory.messages
    # 加上當前人類輸入
    messages = history + [HumanMessage(content=input_text)]
    # 呼叫模型
    response = llm.invoke(messages)
    # 將新對話加入記憶
    memory.chat_memory.add_user_message(input_text)
    memory.chat_memory.add_ai_message(response.content)
    return response.content

# 測試
print("\n---------- 第1次 ----------\n")
print(ask("我叫做王小明，請你記住我的名字!"))

print("\n---------- 第2次 ----------\n")
print(ask("請問我叫做什麼名字?"))



---------- 第1次 ----------

好，王小明，我記住你！ 😊


---------- 第2次 ----------

你叫做小明，正確！ 你是小明嗎？



3.4 利用 Streamlit 快速搭一個對話服務

程式已寫好在`chat_app.py`，請執行`streamlit run chat_app.py`指令。